In [1]:
%pip install psycopg2

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\626\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import psycopg2
from psycopg2 import Error

In [4]:
import psycopg2

def get_connection(database_name):
    # Прописываем функцию для подключения к нашей базе данных
    connection = psycopg2.connect(user="postgres",
                                  password="1",
                                  host="localhost",
                                  port="5432",
                                  database=database_name)
    return connection

def close_connection(connection):
    # Также сразу пишем функцию для закрытия подключения
    if connection:
        connection.close()
        print("Соединение с PostgreSQL закрыто")

try:
    connection = get_connection("pr6")
    cursor = connection.cursor()

    # Создаем таблицу 
    create_table_query = '''
    CREATE TABLE public_transportation_statistics_by_zip_code(
        zip_code character varying(10) NOT NULL PRIMARY KEY,
        public_transportation_pct numeric(15,2) NOT NULL,
        public_transportation_population integer
    );
    '''
    cursor.execute(create_table_query)
    connection.commit()
    print("Таблица 'public_transportation_statistics_by_zip_code' успешно создана")

except (Exception, psycopg2.Error) as error:
    print("Ошибка при подключении или работе с PostgreSQL:", error)

finally:
    # Закрываем подключения к базе данных
    if connection:
        close_connection(connection)

Таблица 'public_transportation_statistics_by_zip_code' успешно создана
Соединение с PostgreSQL закрыто


In [ ]:
import os  
import psycopg2  
from psycopg2 import Error

try:
    connection = get_connection("pr6")
    cursor = connection.cursor()

# Прописываем путь к месту, где хранится CSV-файл
    csv_file_path = r"C:\Users\626\Downloads\public_transportation_statistics_by_zip_code.csv"

    # Проверяем сущестует ли файл
    if not os.path.exists(csv_file_path):
        print(f"ОШИБКА: Файл '{csv_file_path}' не найден.")


        # Вставляем данные из CSV-файла с использованием COPY
    with open(csv_file_path, 'r') as file:
        copy_query = """
        COPY public_transportation_statistics_by_zip_code(zip_code, public_transportation_pct, public_transportation_population)
        FROM STDIN WITH (FORMAT csv, HEADER true, DELIMITER ',');
        """
        cursor.copy_expert(copy_query, file)
        connection.commit()
        print("Данные успешно вставлены в таблицу 'public_transportation_statistics_by_zip_code'")

except (Exception, psycopg2.Error) as error:
    print("Ошибка при подключении или работе с PostgreSQL:", error)

finally:
    # Закрываем подключения к базе данных
    if connection:
        close_connection(connection)

Данные успешно вставлены в таблицу 'public_transportation_statistics_by_zip_code'
Соединение с PostgreSQL закрыто


In [43]:
import psycopg2 
from psycopg2 import Error
try:
    connection = get_connection("pr6")
    cursor = connection.cursor()

    # Составим запрос для того чтобы найти максимальное и минимальное значение процентного соотношения
    query = """
    SELECT 
    MAX(public_transportation_pct) AS max_pct,
    MIN(public_transportation_pct) AS min_pct
    FROM public_transportation_statistics_by_zip_code
    WHERE public_transportation_pct >= 0;
    """
    cursor.execute(query)
    result = cursor.fetchone()
    max_pct, min_pct = result
    print(f"Максимальное процентное соотношение: {max_pct}")
    print(f"Минимальное процентное соотношение: {min_pct}")
    
except (Exception, psycopg2.Error) as error:
    print("Ошибка при подключении или работе с PostgreSQL:", error)

finally:
    # Закрываем подключения к базе данных
    if connection:
        close_connection(connection)


Максимальное процентное соотношение: 100.00
Минимальное процентное соотношение: 0.00
Соединение с PostgreSQL закрыто


In [ ]:
import psycopg2 
from psycopg2 import Error

try:
    connection = get_connection("pr6")
    cursor = connection.cursor()

    # Запишем SQL-запрос для расчета среднего объема продаж
    query = """
SELECT 
    CASE 
        WHEN pts.public_transportation_pct > 10 THEN 'high_transport'
        ELSE 'low_transport'
    END AS "transport_level",
    AVG(s.sales_amount) AS "average sales volume"
    FROM 
    sales s
    JOIN 
    customers c ON s.customer_id = c.customer_id
    JOIN 
    public_transportation_statistics_by_zip_code pts ON c.postal_code = pts.zip_code
    GROUP BY 
    CASE 
        WHEN pts.public_transportation_pct > 10 THEN 'high_transport'
        ELSE 'low_transport'
    END;
    """
    cursor.execute(query)
    results = cursor.fetchall()

# Выводим результат
    print("Уровень использования транспорта | Средний объем продаж")
    for row in results:
        category, avg_sales = row
        print(f"{category} | {avg_sales}")

except (Exception, psycopg2.Error) as error:
    print("Ошибка при подключении или работе с PostgreSQL:", error)

finally:
    # Закрываем подключения к базе данных
    if connection:
        close_connection(connection)


Ошибка при подключении или работе с PostgreSQL: ОШИБКА:  отношение "sales" не существует
LINE 18:     FROM sales s
                  ^

Соединение с PostgreSQL закрыто


In [ ]:
try:
    connection = get_connection("pr6")
    
    # Пишем SQL-запрос для выборки нужных данных
    query = "SELECT * FROM public_transportation_statistics_by_zip_code WHERE public_transportation_pct >= 0;"
    
    # Загрузим данные в DataFrame
    my_data = pandas.read_sql(query, connection)

    # Проведем построение гистограммы распределения
    my_data.plot.hist(y='public_transportation_pct', color='grey')
    plt.title('Гистограмма распределения')
    plt.xlabel('Доля пользователей общественного транспорта')
    plt.ylabel('Частота')
    plt.show()

In [ ]:
try:
    connection = get_connection("pr6")
    
    # Составляем SQL-запрос для группировки данных по zip_code
    query = """SELECT pts.zip_code,
    COUNT(DISTINCT c.customer_id) AS customer_count,
    COUNT(*) / COUNT(DISTINCT c.customer_id) AS avg_transactions,
    AVG(s.sales_amount) AS avg_sales_amount, pts.public_transportation_pct
    FROM 
        customers c
    JOIN 
        sales s ON c.customer_id = s.customer_id
    JOIN 
        public_transportation_statistics_by_zip_code pts ON c.postal_code = pts.zip_code
    WHERE public_transportation_pct >= 0
    GROUP BY pts.zip_code, pts.public_transportation_pct;
    """
    
    # Загрузим данные в DataFrame
    data = pandas.read_sql(query, connection)

    # После экспортируем данные в CSV
    csv_file = "analysis_sales.csv"
    data.to_csv(csv_file, index=False)
    print(f"Данные успешно экспортированы в файл: {csv_file}")

    # Построим диаграмму рассеяния
    plt.scatter(data['public_transportation_pct'], data['avg_sales_amount'], color='grey')
    plt.title('Корреляция между уровнем использования общественного транспорта и объемом продаж')
    plt.xlabel('Доля пользователей общественного транспорта')
    plt.ylabel('Средний объем продаж')
    plt.show()